<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/PyTorch_Convolutional_layers_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -U: Upgrade all packages to the newest available version
!pip install -U d2l

from d2l import torch as d2l
import torch
from torch import nn
import numpy as np

Requirement already up-to-date: d2l in /usr/local/lib/python3.6/dist-packages (0.15.1)


The big part of the code is from d2l.ai book : http://d2l.ai/ ; + experiments with code and some modifications are added

In [ ]:
### input X and kernel K , compute cross correlation
def corr2d(X,K):
  reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
  h,w = K.shape
  Y=torch.zeros(X.shape[0] -h + 1, X.shape[1] - w + 1)
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i,j]=reduce_sum((X[i:i+ h,j:j+w])*K)
  return Y

In [ ]:
X = np.array([[1.0, 2.0,3.0],[2.0,3.0,4.0],[4.0,5.0,6.0]])
K =np.array([[2.0,3.0],[4.0,5.0]])
corr2D(X,K)

tensor([[31., 45.],
        [54., 68.]])

In [ ]:
### 2D Convolutional layer
class conv2D(nn.Module):
  def __init__(self, kernel_size):
    super.__init__()
    self.weight = nn.Parameter(torch.rand(kernel_size))
    self.bias = nn.Parameter(torch.zeros(1))

  def forward(self, x):
    return corr2D(x, self.weight) + self.bias


Edge detection in image

In [ ]:
X = torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [ ]:
### kernel for edge detection
K = torch.tensor([[1.0, -1.0]])

In [ ]:
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [ ]:
transpose = lambda x, *args, **kwargs: x.t(*args, **kwargs)
corr2D(transpose(X),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Learning Kernel to detect edges

In [ ]:
### two dim conv layer with 1 input and 1 output channel, 
### dimention of input is height, weight 
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)
# dimentions : (batch_size, num_channels, height, weight)
X = X.reshape(1,1,6,8)
Y = Y.reshape(1,1,6,7)

In [ ]:
for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i + 1}, loss {l.sum():.3f}')

batch 2, loss 5.623
batch 4, loss 1.146
batch 6, loss 0.276
batch 8, loss 0.080
batch 10, loss 0.027
batch 12, loss 0.010
batch 14, loss 0.004
batch 16, loss 0.002
batch 18, loss 0.001
batch 20, loss 0.000
